# Kanye lyrics with RNN
With this project i attempt to create authentic Kanye lyrics with an RNN.
* The RNN is trained on a Kaggle dataset of Kanye lyrics: https://www.kaggle.com/datasets/convolutionalnn/kanye-west-lyrics-dataset
* The method is based on the TensorFlow Shakespeare text generation tutorial: https://www.tensorflow.org/text/tutorials/text_generation

In [26]:
import tensorflow as tf
import numpy as np
import os
import time

In [27]:
# Read, then decode for py2 compatibility
text = open("/content/Kanye West Lyrics.txt", "rb").read().decode(encoding="utf-8")
# lenght of text is the number of charecters in it
print(f"Length of text: {len(text)} charecters")

Length of text: 365071 charecters


In [28]:
# The unique charecters in the file
vocab = sorted(set(text))
print(f"{len(vocab)} unique charecters")

102 unique charecters


In [29]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab),
                                              mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(),
                                              invert=True,
                                              mask_token=None)

In [30]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [31]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,
                                                  "UTF-8"))
all_ids

<tf.Tensor: shape=(365071,), dtype=int64, numpy=array([102,  57,  33, ...,  16,  16,  16])>

In [32]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [33]:
seq_length = 100

In [34]:
sequences = ids_dataset.batch(seq_length+1,
                              drop_remainder=True)

In [35]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [36]:
dataset = sequences.map(split_input_target)

In [37]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

dataset = (dataset
           .shuffle(BUFFER_SIZE)
           .batch(BATCH_SIZE, drop_remainder=True)
           .prefetch(tf.data.experimental.AUTOTUNE))
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [38]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [39]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                                               embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)

    if states is None:
      states = self.gru.get_initial_state(x)
    
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [41]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [42]:
model = CustomTraining(vocab_size=len(ids_from_chars.get_vocabulary()),
                       embedding_dim=embedding_dim,
                       rnn_units=rnn_units)

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss)

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = "/content/drive/MyDrive/Kanye_logs"
# Name of the checkpoint file
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS = 30

In [46]:
model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
56/56 [==============================] - 9s 82ms/step - loss: 3.5231
Epoch 2/30
56/56 [==============================] - 4s 67ms/step - loss: 2.4571
Epoch 3/30
56/56 [==============================] - 4s 68ms/step - loss: 2.2371
Epoch 4/30
56/56 [==============================] - 4s 65ms/step - loss: 2.0751
Epoch 5/30
56/56 [==============================] - 4s 64ms/step - loss: 1.9234
Epoch 6/30
56/56 [==============================] - 4s 65ms/step - loss: 1.8013
Epoch 7/30
56/56 [==============================] - 4s 70ms/step - loss: 1.6969
Epoch 8/30
56/56 [==============================] - 4s 64ms/step - loss: 1.6040
Epoch 9/30
56/56 [==============================] - 4s 65ms/step - loss: 1.5240
Epoch 10/30
56/56 [==============================] - 4s 63ms/step - loss: 1.4503
Epoch 11/30
56/56 [==============================] - 4s 63ms/step - loss: 1.3778
Epoch 12/30
56/56 [==============================] - 4s 62ms/step - loss: 1.3108
Epoch 13/30
56/56 [==================

In [47]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated
    skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index
        values =[-float("inf")]*len(skip_ids),
        indices=skip_ids,
        # Match the shape of the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs
    input_chars = tf.strings.unicode_split(inputs, "UTF-8")
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model
    # predicted_logits.shape = [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids,
                                          states=states,
                                          return_state=True)
    # Only use the last prediction
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction ma: prevent "[UNK]" from being generated
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs
    predicted_ids = tf.random.categorical(predicted_logits,
                                          num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids,
                               axis=-1)
    
    # Convert from token ids to charecters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the charecters and model state
    return predicted_chars, states

In [48]:
one_step_model = OneStep(model,
                         chars_from_ids,
                         ids_from_chars)

In [59]:
start = time.time()
states = None
next_char = tf.constant(["[Intro]"])
result = [next_char]

for n in range(1200):
  next_char, states = one_step_model.generate_one_step(next_char,
                                                       states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

[Intro]
You popitom, worgh if they'll he's bust there
Everything I wanted, man it supmons
I got play that forever when one of the Lood com
That's high lights (Will hem edopi, baby, we major)
We major? (C'mon, homie, we major)
We major?


[Chorus: Kanye West & The Game]
Close unlems?
You know the family cryin'? A loost by might and Have yauses
(What it means)
Somebody need to pum that that doom is like a man, shoulday
So if you had her too, it don't affect me in the first
But I am nothing like they never ever let your words post
Man I, the kids was like Pick Solo
Oh you're kidains from the same ol'?
The God Capitol packs of the lights
How many cars do we own?000 LL BkNow of God car
Sharter that's purty Dame with mine to make them stays on my diamond-encrote man
Oh, IG, I got your love lockdown
Keeping your love lockdown—your love lockdown
Now keep your love lockdown—you lose


[Outro]
You lose, you never knew
While I'm drol-o, any
You wave your daughter of their fresh
But I can can't th

It's pretty cringe how the text often doesn't make much sense. To combat this I think it would be crucial to implement a module that would only allow for words from a certain set, so that nonense words could be skipped. It would also be an interesting project to introduce structure of the form [INTRO], [VERSE #], ..., [CHORUS], [VERSE #], ..., [OUTRO] so that it could create actual songs.